### Calculate Analyses Values Per Larvae
#### List of tasks accomplished in this Jupyter Notebook:
- Calculate various characteristics of larval behavior including P, CS, CTI, etc.
- Stack the acclimation and experiment dataframes horizontally for further analysis
- After reviewing the preferences of larvae for each odor, assign each odor as Aversive, Apetitive, or Neutral. 

In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
df = pd.read_csv("./data/experiment_IDs/cleaned_static_data.csv")

animals = df["animal_ID"].unique()
# Check that animal IDs in dataframe are unique
assert len(df) == len(animals)

In [3]:
def safe_divide(x, y):
    x = safe_get(x)
    y = safe_get(y)
    try: 
        ans = x / y
    except ZeroDivisionError: 
        return 0
    return ans
    
def safe_get(x, ans=0):
    if math.isnan(x):
        return ans
    return x

- Calculate various characteristics of larval behavior including P, CS, CTI, etc.

In [4]:
for val in ["acclimate", "experiment"]:
    master_df = pd.DataFrame()

    for index, row in df.iterrows():
        filename = "./data/trajectories/video_calculations/"+row["animal_ID"]+"-"+val+".csv"
        
        temp = pd.read_csv(filename)
        temp_move = temp[temp["moving"] == True]
        
        ############# CALCULATIONS OVER ALL DATA POINTS #############
        
        # MEDIAN CONCENTRATION -----------------------------
        median_conc = safe_get(temp["concentration"].median())

        # MEDIAN SPEED IN BODY LENGTHS ---------------------
        median_speed = safe_divide(temp_move["speed_mm_s"].median(), row["larvae_length_mm"])
        
        # % TIME SPENT MOVING ------------------------------
        time_move = safe_divide(len(temp_move), len(temp))

        # % TIME SPENT NEXT TO AT LEAST 1 WALL -------------
        time_wall = safe_get(temp["wall"].mean())
        
        # DISCOVERY TIME IN SECONDS ------------------------
        if len(temp[temp["concentration"] >= 50]) > 0:
            discovery_time = safe_divide(temp[temp["concentration"] >= 50]["frames"].values[0], 2) # 2 fps
        else:
            discovery_time = 900
        
        ############# CALCULATIONS OVER CONCENTRATION THRESHOLDS #############
        
        # Divide up into thresholds for concentration and concentration differences
        high = temp[temp["concentration"] >= 50]
        low = temp[temp["concentration"] < 50]
        up = temp[temp["concentration_delta"] >= 1]
        down = temp[temp["concentration_delta"] <= -1]
        
        high_move = high[high["moving"] == True]
        low_move = low[low["moving"] == True]
        up_move = up[up["moving"] == True]
        down_move = down[down["moving"] == True]

        # % TIME SPENT MOVING UP/DOWN ----------------------
        # concentration change dependent
        cd_move = safe_divide(len(up_move)-len(down_move), len(up_move)+len(down_move))

        # SPEED IN BODY LENGTHS ----------------------------
        # concentration change dependent
        up_speed = safe_divide(up_move["speed_mm_s"].median(), row["larvae_length_mm"])
        down_speed = safe_divide(down_move["speed_mm_s"].median(), row["larvae_length_mm"])
        cd_speed = up_speed - down_speed
        # concentration dependent
        high_speed = safe_divide(high_move["speed_mm_s"].median(), row["larvae_length_mm"])
        low_speed = safe_divide(low_move["speed_mm_s"].median(), row["larvae_length_mm"])
        c_speed = high_speed - low_speed

        # TURN INCIDENCE -----------------------------------
        # concentration change dependent
        up_turns = safe_get(up_move["turn"].mean())
        down_turns = safe_get(down_move["turn"].mean())
        cd_turn = up_turns - down_turns
        # concentration dependent
        high_turns = safe_get(high_move["turn"].mean())
        low_turns = safe_get(low_move["turn"].mean())
        c_turn = high_turns - low_turns

        temp = pd.DataFrame({# STATIC VARIABLES --------------------------
                            "animal_ID": [row["animal_ID"]], 
                            "treatment_odor": row["treatment_odor"],
                            "larvae_length_mm": row["larvae_length_mm"], 
                            "minutes_past_L": row["minutes_past_L"], 
                            "starved": row["starved"],
                            "sex": row["sex"], 

                            # VARIABLES CALCULATED FOR ENTIRE TIME -------
                            "median_conc": median_conc, 
                            "median_speed": median_speed,
                            "time_move": time_move,
                            "time_wall": time_wall, 
                            "discovery_time": discovery_time,

                            # CONCENTRATION VARIABLES --------------------
                            "cd_move": cd_move, 
                            "cd_speed": cd_speed, 
                            "c_speed": c_speed, 
                            "cd_turn": cd_turn, 
                            "c_turn": c_turn

                             })
        master_df = pd.concat([master_df, temp])

    master_df.to_csv('./data/trajectories/summary/cleaned_animal_analyses_'+val+'.csv', index=False)

- Stack the acclimation and experiment dataframes horizontally for further analysis

In [5]:
acc = pd.read_csv('./data/trajectories/summary/cleaned_animal_analyses_acclimate.csv')
exp = pd.read_csv('./data/trajectories/summary/cleaned_animal_analyses_experiment.csv')

# Remove static data from one of the dataframes
exp.drop(["sex", "larvae_length_mm", "minutes_past_L", "starved", "treatment_odor"], inplace=True, axis=1)

# Rename the dataframe columns to mark acclimation or experiment
acc.columns = "A_"+acc.columns
exp.columns = "E_"+exp.columns

assert len(acc) == len(exp)
assert set(acc["A_animal_ID"]) == set(exp["E_animal_ID"])

df = pd.concat([acc.set_index("A_animal_ID"), exp.set_index("E_animal_ID")], axis=1)
assert len(df) == len(acc)

df["median_conc_diff"] = df["E_median_conc"] - df["A_median_conc"] 
df["c_speed_diff"] = df["E_c_speed"] - df["A_c_speed"] 
df["c_turn_diff"] = df["E_c_turn"] - df["A_c_turn"] 
df["cd_turn_diff"] = df["E_cd_turn"] - df["A_cd_turn"] 
df["cd_move_diff"] = df["E_cd_move"] - df["A_cd_move"] 
df["cd_speed_diff"] = df["E_cd_speed"] - df["A_cd_speed"] 
df["discovery_time_diff"] = df["E_discovery_time"] - df["A_discovery_time"] 

df.to_csv('./data/trajectories/summary/cleaned_animal_analyses.csv', index=False)

- After reviewing the preferences of larvae for each odor, assign each odor as Aversive, Apetitive, or Neutral. 

In [6]:
df = pd.read_csv("./data/trajectories/summary/cleaned_animal_analyses.csv")
df["stimulus"] = df["A_treatment_odor"]

# Starved animals
df_str = df[df["A_starved"] == '1day'].copy()
df_str["stimulus"] = df_str["stimulus"].replace({"naive_100ul_left_quinine_10mM": "Aversive", 
                                         "naive_100ul_left_food_extract": "Appetitive",
                                         "naive_100ul_left_indole_10mM": "Neutral", 
                                         "naive_100ul_left_food_05percent": "Appetitive",
                                         "naive_100ul_left_indole_100uM": "Neutral",
                                         "naive_100ul_left_milliQ_water": "Neutral",
                                         "naive_100ul_left_o-cresol_100uM": "Neutral",
                                         "naive_100ul_left_yeastRNA_1gL": "Appetitive",
                                         "naive_100ul_left_glucose_10gL": "Neutral",
                                         "naive_100ul_left_amino_acids": "Neutral"
                                        })

# Fed animals
df_fed = df[df["A_starved"] == 'no'].copy()
df_fed["stimulus"] = df_fed["stimulus"].replace({"naive_100ul_left_quinine_10mM": "Aversive", 
                                         "naive_100ul_left_food_extract": "Neutral",
                                         "naive_100ul_left_indole_10mM": "Neutral", 
                                         "naive_100ul_left_food_05percent": "Neutral",
                                         "naive_100ul_left_indole_100uM": "Neutral",
                                         "naive_100ul_left_milliQ_water": "Neutral",
                                         "naive_100ul_left_o-cresol_100uM": "Aversive",
                                         "naive_100ul_left_yeastRNA_1gL": "Neutral",
                                         "naive_100ul_left_glucose_10gL": "Neutral",
                                         "naive_100ul_left_amino_acids": "Neutral"
                                        })

df = pd.concat([df_fed, df_str])
df.to_csv("./data/trajectories/summary/cleaned_animal_analyses_stimuli_groups.csv", index=False)